In [54]:
import pandas as pd

# Load the dataset
file_path = 'C:/Users/cueva/Downloads/Copy of Marketing Data - data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

,Date,Platform,Campaign,Region,Spend,CPM,Impressions,Frequency,Clicks,Purchases,Revenue,Product_Category,Target_Audience,Creative_Type,Video_Completion_Rate,Customer_LTV,Is_Competitive_Event
0,1/1/2024,FB,FB_Athletes_Video_Protein_001,West,1187.29,18.0,65960,2.75,527,15,1216.74,Protein,Athletes,Video,0.486,772.8,True
1,1/1/2024,FB,FB_Athletes_Video_Protein_001,South,1081.89,18.0,60105,2.51,512,12,868.16,Protein,Athletes,Video,0.486,772.8,True
2,1/1/2024,FB,FB_Athletes_Video_Protein_001,Northeast,648.47,18.0,36025,2.52,344,9,694.68,Protein,Athletes,Video,0.486,772.8,True
3,1/1/2024,FB,FB_Athletes_Video_Protein_001,Midwest,463.38,18.0,25743,2.49,272,6,482.66,Protein,Athletes,Video,0.486,772.8,True
4,1/1/2024,FB,FB_Athletes_Image_Preworkout_002,West,800.59,18.0,44477,3.10,418,14,852.22,Preworkout,Athletes,Image,NaN,538.2,True


In [55]:
# Check the number of rows and columns in the dataset
rows, columns = data.shape
rows, columns

(3240, 17)

In [56]:
# Check the data types of each column
column_types = data.dtypes
column_types

Date                      object
Platform                  object
Campaign                  object
Region                    object
Spend                    float64
CPM                      float64
Impressions                int64
Frequency                float64
Clicks                     int64
Purchases                  int64
Revenue                  float64
Product_Category          object
Target_Audience           object
Creative_Type             object
Video_Completion_Rate    float64
Customer_LTV             float64
Is_Competitive_Event        bool
dtype: object

In [57]:
# Convert the Date column to datetime format using the correct DD/MM/YYYY format
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y', errors='coerce')

# Verify the conversion and check for invalid dates
invalid_date_count = data['Date'].isnull().sum()
data['Date'].head(), invalid_date_count

# Calculate the percentage of missing values
missing_percent = (data.isnull().sum() / len(data)) * 100

# Combine into a DataFrame for easier interpretation
missing_summary = pd.DataFrame({
    'Missing Values': data.isnull().sum(),
    'Percentage (%)': missing_percent
})

# Display the summary
print(missing_summary)

                       Missing Values  Percentage (%)
Date                                0        0.000000
Platform                            0        0.000000
Campaign                            0        0.000000
Region                              0        0.000000
Spend                               0        0.000000
CPM                                 0        0.000000
Impressions                         0        0.000000
Frequency                           0        0.000000
Clicks                              0        0.000000
Purchases                           0        0.000000
Revenue                            20        0.617284
Product_Category                    0        0.000000
Target_Audience                     0        0.000000
Creative_Type                       0        0.000000
Video_Completion_Rate            2012       62.098765
Customer_LTV                        0        0.000000
Is_Competitive_Event                0        0.000000


In [58]:
# Step 1: Filter the data where Creative_Type is not "Video" and ensure Video_Completion_Rate is 0
data.loc[data['Creative_Type'] != 'Video', 'Video_Completion_Rate'] = 0

# Step 2: Calculate the mean and median for Video_Completion_Rate for rows with Creative_Type as "Video"
mean_value = data.loc[data['Creative_Type'] == 'Video', 'Video_Completion_Rate'].mean()
median_value = data.loc[data['Creative_Type'] == 'Video', 'Video_Completion_Rate'].median()

mean_value, median_value

(np.float64(0.5749983713355048), np.float64(0.5896))

In [59]:
# Apply median imputation for missing values in Video_Completion_Rate
median_value = data.loc[data['Creative_Type'] == 'Video', 'Video_Completion_Rate'].median()
data['Video_Completion_Rate'].fillna(median_value, inplace=True)

# Verify if all missing values in the column have been handled
missing_values_after_imputation = data['Video_Completion_Rate'].isnull().sum()
missing_values_after_imputation

C:\Users\cueva\AppData\Local\Temp\ipykernel_20264\791920204.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Video_Completion_Rate'].fillna(median_value, inplace=True)


np.int64(0)

In [60]:
# Check for missing values in each column
missing_values = data.isnull().sum()

# Calculate the percentage of missing values
missing_percent = (missing_values / len(data)) * 100

# Combine into a summary DataFrame for review
missing_summary = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percent
})

# Display the summary
print(missing_summary)

                       Missing Values  Percentage (%)
Date                                0        0.000000
Platform                            0        0.000000
Campaign                            0        0.000000
Region                              0        0.000000
Spend                               0        0.000000
CPM                                 0        0.000000
Impressions                         0        0.000000
Frequency                           0        0.000000
Clicks                              0        0.000000
Purchases                           0        0.000000
Revenue                            20        0.617284
Product_Category                    0        0.000000
Target_Audience                     0        0.000000
Creative_Type                       0        0.000000
Video_Completion_Rate               0        0.000000
Customer_LTV                        0        0.000000
Is_Competitive_Event                0        0.000000


In [61]:
#################### IMPUTATION OF MISSING VALUES IN REVENUE BASED ON FILLED ROWS ################

# Step 1: Calculate the average revenue per purchase for rows with non-missing Revenue
# Only include rows where Revenue > 0 and Purchases > 0
valid_revenue_data = data[(data['Revenue'] > 0) & (data['Purchases'] > 0)]
avg_revenue_per_purchase = valid_revenue_data['Revenue'].sum() / valid_revenue_data['Purchases'].sum()

# Step 2: Identify rows with missing Revenue before imputation
missing_revenue_indices = data[data['Revenue'].isnull()].index

# Step 3: Impute missing Revenue values based on Purchases
# Ensure Purchases is greater than 0 for imputation to avoid invalid computations
data.loc[data['Revenue'].isnull() & (data['Purchases'] > 0), 'Revenue'] = (
    data.loc[data['Revenue'].isnull() & (data['Purchases'] > 0), 'Purchases'] * avg_revenue_per_purchase
)

# Step 4: Display the rows where missing Revenue was filled
filled_rows = data.loc[missing_revenue_indices]
print(filled_rows)

           Date Platform                                   Campaign  \
194  2024-01-06   Google         Google_Athletes_Search_Protein_001   
589  2024-01-17   Google         Google_Athletes_Search_Protein_001   
1413 2024-02-09       FB     FB_FitnessEnth_Carousel_WeightLoss_001   
1658 2024-02-16       FB              FB_Athletes_Video_Protein_001   
1661 2024-02-16       FB           FB_Athletes_Image_Preworkout_002   
1682 2024-02-16       TT              TT_Athletes_Video_Protein_001   
1738 2024-02-18       FB     FB_FitnessEnth_Carousel_WeightLoss_001   
1881 2024-02-22       FB     FB_FitnessEnth_Carousel_WeightLoss_001   
1912 2024-02-23       FB           FB_Athletes_Image_Preworkout_002   
1947 2024-02-24       FB              FB_Athletes_Video_Protein_001   
2146 2024-02-29   Google  Google_FitnessEnth_Display_Preworkout_001   
2148 2024-02-29       TT              TT_Athletes_Video_Protein_001   
2557 2024-03-12       FB              FB_Athletes_Video_Protein_001   
2678 2

In [62]:
# Check for missing values in each column
missing_values = data.isnull().sum()

# Calculate the percentage of missing values
missing_percent = (missing_values / len(data)) * 100

# Combine into a summary DataFrame for easier review
missing_summary = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percent
})

# Display the summary
print(missing_summary)

                       Missing Values  Percentage (%)
Date                                0             0.0
Platform                            0             0.0
Campaign                            0             0.0
Region                              0             0.0
Spend                               0             0.0
CPM                                 0             0.0
Impressions                         0             0.0
Frequency                           0             0.0
Clicks                              0             0.0
Purchases                           0             0.0
Revenue                             0             0.0
Product_Category                    0             0.0
Target_Audience                     0             0.0
Creative_Type                       0             0.0
Video_Completion_Rate               0             0.0
Customer_LTV                        0             0.0
Is_Competitive_Event                0             0.0


In [63]:
data.describe()

,Date,Spend,CPM,Impressions,Frequency,Clicks,Purchases,Revenue,Video_Completion_Rate,Customer_LTV
count,3240,3240.000000,3240.000000,3240.000000,3240.000000,3240.000000,3240.000000,3240.000000,3240.000000,3240.000000
mean,2024-02-14 12:00:00.000000256,1495.577633,14.223593,100056.933951,3.008373,687.963889,19.274383,1293.917227,0.256510,538.660000
min,2024-01-01 00:00:00,326.510000,10.010000,0.000000,2.010000,0.000000,0.000000,113.600000,0.000000,252.720000
25%,2024-01-23 00:00:00,981.400000,12.160000,63965.000000,2.490000,409.000000,10.000000,646.080000,0.000000,393.120000
50%,2024-02-14 12:00:00,1261.495000,14.180000,88889.500000,2.850000,611.000000,16.000000,1064.485000,0.000000,538.200000
75%,2024-03-08 00:00:00,1659.267500,15.760000,126429.500000,3.362500,880.000000,24.000000,1708.230000,0.589600,604.800000
max,2024-03-30 00:00:00,119315.000000,23.850000,365425.000000,7.780000,2866.000000,107.000000,6987.780000,0.741000,940.800000
std,NaN,3727.564711,2.989872,50143.396744,0.702799,413.818091,13.797493,880.999683,0.292442,202.036512


In [64]:
############# INCONSISTENT DATA - NO CLICKS OR PURCHASES BUT THERE'S A VALUE IN THE REVENUE COLUMN ##############
inconsistent_rows = data[(data['Purchases'] == 0) & (data['Revenue'] > 0)]
print(inconsistent_rows)
print(f"Number of inconsistent rows: {inconsistent_rows.shape[0]}")
# Removed the inconsistent rows
data = data[~((data['Purchases'] == 0) & (data['Revenue'] > 0))]

           Date Platform                                   Campaign   Region  \
51   2024-01-02   Google         Google_Athletes_Search_Protein_001  Midwest   
199  2024-01-06   Google          Google_WeightLoss_Search_Diet_001  Midwest   
251  2024-01-07       TT               TT_WeightLoss_Video_Diet_001  Midwest   
259  2024-01-08       FB           FB_Athletes_Image_Preworkout_002  Midwest   
311  2024-01-09   Google  Google_FitnessEnth_Display_Preworkout_001  Midwest   
615  2024-01-18       FB              FB_Athletes_Video_Protein_001  Midwest   
639  2024-01-18       TT              TT_Athletes_Video_Protein_001  Midwest   
655  2024-01-19       FB           FB_Athletes_Image_Preworkout_002  Midwest   
679  2024-01-19       TT        TT_FitnessEnth_Video_Preworkout_001  Midwest   
887  2024-01-25   Google  Google_FitnessEnth_Display_Preworkout_001  Midwest   
907  2024-01-26       FB           FB_Athletes_Image_Preworkout_002  Midwest   
943  2024-01-27       FB           FB_At

In [65]:
# Add calculated columns for baseline metrics
data['ROAS'] = data['Revenue'] / data['Spend']  # Return on Ad Spend
data['CTR'] = data['Clicks'] / data['Impressions']  # Click-Through Rate
data['CVR'] = data['Purchases'] / data['Clicks']  # Conversion Rate
data['CPC'] = data['Spend'] / data['Clicks']  # Cost per Click
data['CPP'] = data['Spend'] / data['Purchases']  # Cost per Purchase
data['Impressions_per_Dollar'] = data['Impressions'] / data['Spend']  # Impressions per Dollar

# Handling divisions by zero or missing data
data.replace([float('inf'), -float('inf')], None, inplace=True)  # Replace infinite values
data.fillna(0, inplace=True)  # Replace NaN with 0 for simplicity

# Verify the added columns
print(data.head())

        Date Platform                          Campaign     Region    Spend  \
0 2024-01-01       FB     FB_Athletes_Video_Protein_001       West  1187.29   
1 2024-01-01       FB     FB_Athletes_Video_Protein_001      South  1081.89   
2 2024-01-01       FB     FB_Athletes_Video_Protein_001  Northeast   648.47   
3 2024-01-01       FB     FB_Athletes_Video_Protein_001    Midwest   463.38   
4 2024-01-01       FB  FB_Athletes_Image_Preworkout_002       West   800.59   

    CPM  Impressions  Frequency  Clicks  Purchases  ...  Creative_Type  \
0  18.0        65960       2.75     527         15  ...          Video   
1  18.0        60105       2.51     512         12  ...          Video   
2  18.0        36025       2.52     344          9  ...          Video   
3  18.0        25743       2.49     272          6  ...          Video   
4  18.0        44477       3.10     418         14  ...          Image   

  Video_Completion_Rate Customer_LTV Is_Competitive_Event      ROAS       CTR  \

C:\Users\cueva\AppData\Local\Temp\ipykernel_20264\308997697.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(0, inplace=True)  # Replace NaN with 0 for simplicity


In [66]:
# Calculate summary statistics for all numerical columns
summary_statistics = data.describe(include='all')

# Display the summary
print(summary_statistics)

                                 Date Platform  \
count                            3193     3193   
unique                            NaN        3   
top                               NaN       TT   
freq                              NaN     1068   
mean    2024-02-14 09:46:16.949577216      NaN   
min               2024-01-01 00:00:00      NaN   
25%               2024-01-23 00:00:00      NaN   
50%               2024-02-14 00:00:00      NaN   
75%               2024-03-08 00:00:00      NaN   
max               2024-03-30 00:00:00      NaN   
std                               NaN      NaN   

                                      Campaign Region          Spend  \
count                                     3193   3193    3193.000000   
unique                                       9      4            NaN   
top     FB_FitnessEnth_Carousel_WeightLoss_001   West            NaN   
freq                                       359    810            NaN   
mean                                   

In [67]:
data.to_csv('C:/Users/cueva/Downloads/cleaned_marketing_data.csv', index=False)

In [68]:
# List of categorical columns
categorical_columns = data.select_dtypes(include=['object', 'category']).columns

# Unique categories in each categorical column
categories_summary = {col: data[col].unique().tolist() for col in categorical_columns}

categories_summary

{'Platform': ['FB', 'Google', 'TT'],
 'Campaign': ['FB_Athletes_Video_Protein_001',
  'FB_Athletes_Image_Preworkout_002',
  'FB_FitnessEnth_Carousel_WeightLoss_001',
  'Google_Athletes_Search_Protein_001',
  'Google_WeightLoss_Search_Diet_001',
  'Google_FitnessEnth_Display_Preworkout_001',
  'TT_Athletes_Video_Protein_001',
  'TT_FitnessEnth_Video_Preworkout_001',
  'TT_WeightLoss_Video_Diet_001'],
 'Region': ['West', 'South', 'Northeast', 'Midwest'],
 'Product_Category': ['Protein', 'Preworkout', 'WeightLoss', 'Diet'],
 'Target_Audience': ['Athletes', 'FitnessEnth', 'WeightLoss'],
 'Creative_Type': ['Video', 'Image', 'Carousel', 'Search', 'Display']}

In [69]:
# Check for missing values in each column
missing_values = data.isnull().sum()

# Calculate the percentage of missing values
missing_percent = (missing_values / len(data)) * 100

# Combine into a summary DataFrame for easier review
missing_summary = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percent
})

# Display the summary
print(missing_summary)

                        Missing Values  Percentage (%)
Date                                 0             0.0
Platform                             0             0.0
Campaign                             0             0.0
Region                               0             0.0
Spend                                0             0.0
CPM                                  0             0.0
Impressions                          0             0.0
Frequency                            0             0.0
Clicks                               0             0.0
Purchases                            0             0.0
Revenue                              0             0.0
Product_Category                     0             0.0
Target_Audience                      0             0.0
Creative_Type                        0             0.0
Video_Completion_Rate                0             0.0
Customer_LTV                         0             0.0
Is_Competitive_Event                 0             0.0
ROAS      

In [70]:
from scipy.stats import shapiro, normaltest

# Select numerical columns
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns

# Initialize a list to store normality results
normality_results = []

# Loop through each numerical column
for column in numerical_columns:
    # Perform the Shapiro-Wilk test for normality
    stat, p_value = shapiro(data[column].dropna())
    
    # Perform the D'Agostino and Pearson's test for normality
    dagostino_stat, dagostino_p = normaltest(data[column].dropna())
    
    # Determine if the column is normally distributed based on p-values
    shapiro_normal = p_value > 0.05
    dagostino_normal = dagostino_p > 0.05
    
    # Append the results to the list
    normality_results.append({
        'Column': column,
        'Shapiro-Wilk Statistic': stat,
        'Shapiro-Wilk p-value': p_value,
        'Shapiro-Wilk Normal': shapiro_normal,
    })

# Convert results to a DataFrame for display
normality_results_df = pd.DataFrame(normality_results)

# Display the results
print(normality_results_df)


                    Column  Shapiro-Wilk Statistic  Shapiro-Wilk p-value  \
0                    Spend                0.067029          1.221546e-82   
1                      CPM                0.939958          3.784967e-34   
2              Impressions                0.923670          1.599141e-37   
3                Frequency                0.900552          1.957969e-41   
4                   Clicks                0.922254          8.681083e-38   
5                Purchases                0.846104          2.473995e-48   
6                  Revenue                0.885567          1.362799e-43   
7    Video_Completion_Rate                0.720301          1.151716e-58   
8             Customer_LTV                0.913044          2.004235e-39   
9                     ROAS                0.911486          1.093946e-39   
10                     CTR                0.924567          2.365123e-37   
11                     CVR                0.983171          4.866049e-19   
12          

In [71]:
import pandas as pd
from scipy.stats import kruskal

# Calculate performance metrics for each row
data['CTR'] = data['Clicks'] / data['Impressions']
data['CPC'] = data['Spend'] / data['Clicks']
data['CVR'] = data['Purchases'] / data['Clicks']
data['ROAS'] = data['Revenue'] / data['Spend']

# Handle divisions by zero or missing data
data.replace([float('inf'), -float('inf')], None, inplace=True)  # Replace infinite values
data.fillna(0, inplace=True)  # Replace NaN with 0 for simplicity

# Aggregate metrics by Platform and compute only medians
channel_metrics = data.groupby('Platform').agg({
    'CTR': 'median',
    'CPC': 'median',
    'CVR': 'median',
    'ROAS': 'median'
}).reset_index()

channel_metrics.columns = ['Platform', 'CTR_Median', 'CPC_Median', 'CVR_Median', 'ROAS_Median']

# Perform Kruskal-Wallis test for each metric
kruskal_results = {}
metrics = ['CTR', 'CPC', 'CVR', 'ROAS']

for metric in metrics:
    groups = [data[data['Platform'] == platform][metric].dropna() for platform in data['Platform'].unique()]
    stat, p_value = kruskal(*groups)
    kruskal_results[metric] = {'Kruskal-Wallis Statistic': stat, 'p-value': p_value}

# Display results
print("Channel Performance Metrics:")
print(channel_metrics)

print("\nKruskal-Wallis Test Results:")
for metric, result in kruskal_results.items():
    print(f"{metric}: Statistic = {result['Kruskal-Wallis Statistic']}, p-value = {result['p-value']}")


Channel Performance Metrics:
  Platform  CTR_Median  CPC_Median  CVR_Median  ROAS_Median
0       FB    0.005560    2.508365    0.027778     0.748881
1   Google    0.009801    1.516850    0.024283     0.943967
2       TT    0.006149    1.845897    0.025799     0.861996

Kruskal-Wallis Test Results:
CTR: Statistic = 636.6311863943273, p-value = 5.718661239781508e-139
CPC: Statistic = 539.0090414121217, p-value = 9.029710139915002e-118
CVR: Statistic = 148.99027791345486, p-value = 4.43784594889025e-33
ROAS: Statistic = 104.57364475947179, p-value = 1.959393054328742e-23


C:\Users\cueva\AppData\Local\Temp\ipykernel_20264\3093681422.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(0, inplace=True)  # Replace NaN with 0 for simplicity


In [72]:
import pandas as pd
from scipy.stats import mannwhitneyu
import itertools

# Calculate aggregated median values for each region
regional_metrics = data.groupby('Region').agg({
    'CTR': 'median',
    'CPC': 'median',
    'CVR': 'median',
    'ROAS': 'median'
}).reset_index()

# Rename columns for clarity
regional_metrics.columns = ['Region', 'CTR_Median', 'CPC_Median', 'CVR_Median', 'ROAS_Median']

# Perform pairwise Mann-Whitney U tests for each metric across regions
metrics = ['CTR', 'CPC', 'CVR', 'ROAS']
regions = data['Region'].unique()
pairwise_tests = []

for metric in metrics:
    for region1, region2 in itertools.combinations(regions, 2):
        # Select median values for each region and metric
        group1 = data[data['Region'] == region1][f'{metric}'].dropna()
        group2 = data[data['Region'] == region2][f'{metric}'].dropna()
        
        # Perform Mann-Whitney U Test (two-sided)
        stat, p_value = mannwhitneyu(group1, group2, alternative='two-sided')
        
        # Append results
        pairwise_tests.append({
            'Metric': metric,
            'Region 1': region1,
            'Region 2': region2,
            'Mann-Whitney U Statistic': stat,
            'p-value': p_value
        })

# Convert pairwise test results to DataFrame
pairwise_tests_df = pd.DataFrame(pairwise_tests)

# Display regional performance metrics (only median)
print("\nRegional Performance Metrics (Median Only):")
print(regional_metrics)

# Display pairwise Mann-Whitney U test results
print("\nMann-Whitney U Test Results (Pairwise Regional Comparisons):")
print(pairwise_tests_df)



Regional Performance Metrics (Median Only):
      Region  CTR_Median  CPC_Median  CVR_Median  ROAS_Median
0    Midwest    0.006557    1.948889    0.023327     0.692747
1  Northeast    0.006390    1.949343    0.028381     0.867126
2      South    0.006644    1.973349    0.026838     0.852492
3       West    0.006635    1.948923    0.026342     0.888885

Mann-Whitney U Test Results (Pairwise Regional Comparisons):
   Metric   Region 1   Region 2  Mann-Whitney U Statistic       p-value
0     CTR       West      South                  325794.0  8.106512e-01
1     CTR       West  Northeast                  333637.0  5.528986e-01
2     CTR       West    Midwest                  309801.0  9.304775e-01
3     CTR      South  Northeast                  335836.0  4.082350e-01
4     CTR      South    Midwest                  312044.5  7.365512e-01
5     CTR  Northeast    Midwest                  304734.0  6.344800e-01
6     CPC       West      South                  324547.0  7.098584e-01
7     C

In [73]:
import pandas as pd
from scipy.stats import chi2_contingency, spearmanr

# Calculate metrics for analysis
data['CTR'] = data['Clicks'] / data['Impressions']
data['CVR'] = data['Purchases'] / data['Clicks']

# Handle missing and infinite values
data.replace([float('inf'), -float('inf')], None, inplace=True)
data.fillna(0, inplace=True)

# Group data by Creative_Type and calculate median CTR, CVR, and Video_Completion_Rate
creative_performance = data.groupby('Creative_Type').agg({
    'CTR': 'median',
    'CVR': 'median',
    'Video_Completion_Rate': 'median',
    'Purchases': 'sum'
}).reset_index()

# Chi-square test: Evaluate relationship between Creative_Type and Purchases
creative_purchases = pd.crosstab(data['Creative_Type'], data['Purchases'] > 0)
chi2_stat, p_value, dof, expected = chi2_contingency(creative_purchases)

# Spearman correlation: Assess Video_Completion_Rate and Purchases
video_data = data[data['Creative_Type'] == 'Video']
spearman_corr, spearman_p = spearmanr(video_data['Video_Completion_Rate'], video_data['Purchases'])

# Display results
print("Creative Performance Metrics:")
print(creative_performance)

print("\nChi-Square Test Results:")
print(f"Chi-Square Statistic = {chi2_stat}, p-value = {p_value}")

print("\nSpearman Correlation (Video Completion Rate vs. Purchases):")
print(f"Spearman Correlation = {spearman_corr}, p-value = {spearman_p}")

Creative Performance Metrics:
  Creative_Type       CTR       CVR  Video_Completion_Rate  Purchases
0      Carousel  0.005140  0.028926                 0.0000       7465
1       Display  0.009774  0.016372                 0.0000       3755
2         Image  0.005190  0.025078                 0.0000       3749
3        Search  0.009893  0.027778                 0.0000      16864
4         Video  0.006113  0.026670                 0.5896      30616

Chi-Square Test Results:
Chi-Square Statistic = 0.0, p-value = 1.0

Spearman Correlation (Video Completion Rate vs. Purchases):
Spearman Correlation = 0.17840753364879702, p-value = 1.2341201658051245e-11


C:\Users\cueva\AppData\Local\Temp\ipykernel_20264\4034706636.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(0, inplace=True)


In [74]:
# Convert Is_Competitive_Event column to numerical (1 for True, 0 for False)
data['Is_Competitive_Event'] = data['Is_Competitive_Event'].astype(int)

# Proceed with the analysis as previously outlined


from scipy.stats import wilcoxon

# Create subsets for days with and without competitive events
competitive_days = data[data['Is_Competitive_Event'] == 1]
non_competitive_days = data[data['Is_Competitive_Event'] == 0]

# Metrics to analyze
metrics = ['CTR', 'CVR', 'ROAS', 'Revenue']
impact_results = []

# Perform Wilcoxon signed-rank test for each metric
for metric in metrics:
    # Ensure equal number of observations for comparison
    competitive_values = competitive_days[metric].dropna()
    non_competitive_values = non_competitive_days[metric].dropna()
    common_length = min(len(competitive_values), len(non_competitive_values))
    
    if common_length > 0:
        stat, p_value = wilcoxon(
            competitive_values[:common_length], 
            non_competitive_values[:common_length]
        )
        impact_results.append({
            'Metric': metric,
            'Wilcoxon Statistic': stat,
            'p-value': p_value
        })

# Evaluate revenue trends (before, during, and after competitive events)
data['Event_Period'] = data['Is_Competitive_Event'].apply(
    lambda x: 'During' if x == 1 else 'Non-Competitive'
)
revenue_trends = data.groupby('Event_Period')['Revenue'].mean().reset_index()

# Display results
impact_results_df = pd.DataFrame(impact_results)
print("Wilcoxon Test Results (Impact of Competitive Events):")
print(impact_results_df)

print("\nRevenue Trends (Before, During, After Events):")
print(revenue_trends)


Wilcoxon Test Results (Impact of Competitive Events):
    Metric  Wilcoxon Statistic       p-value
0      CTR              2346.0  2.690160e-08
1      CVR              4435.0  1.914253e-01
2     ROAS               359.0  7.477187e-22
3  Revenue               706.0  5.572187e-19

Revenue Trends (Before, During, After Events):
      Event_Period      Revenue
0           During  1077.909859
1  Non-Competitive  1311.244826


In [75]:
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

# Analyze the relationship between Frequency and CVR
data['CVR'] = data['Purchases'] / data['Clicks']

# Handle missing and infinite values
data.replace([float('inf'), -float('inf')], None, inplace=True)
data.fillna(0, inplace=True)

# Spearman correlation analysis
spearman_corr, spearman_p = spearmanr(data['Frequency'], data['CVR'])

# Identify trends in Frequency and CVR
frequency_bins = data.groupby(pd.cut(data['Frequency'], bins=[0, 2, 4, 5, 10, float('inf')])).agg({
    'CVR': 'mean'
}).reset_index()

# Display results
print("Spearman Correlation Results:")
print(f"Spearman Correlation = {spearman_corr}, p-value = {spearman_p}")

print("\nTrends in Frequency and CVR (Binned Data):")
print(frequency_bins)


Spearman Correlation Results:
Spearman Correlation = 0.003805842299440415, p-value = 0.8297892101019277

Trends in Frequency and CVR (Binned Data):
     Frequency       CVR
0   (0.0, 2.0]       NaN
1   (2.0, 4.0]  0.025706
2   (4.0, 5.0]  0.026329
3  (5.0, 10.0]  0.023799
4  (10.0, inf]       NaN


C:\Users\cueva\AppData\Local\Temp\ipykernel_20264\3112119706.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(0, inplace=True)
C:\Users\cueva\AppData\Local\Temp\ipykernel_20264\3112119706.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  frequency_bins = data.groupby(pd.cut(data['Frequency'], bins=[0, 2, 4, 5, 10, float('inf')])).agg({
